# Folio Executors: Automated Experiment Loops

This notebook demonstrates how to use **Executors** in Folio for automated
closed-loop optimization. Executors bridge the gap between Folio's suggestions
and actual experiment execution.

We'll cover:
1. What executors are and why they're useful
2. Creating a custom executor for simulation
3. Using `folio.execute()` for automated optimization loops
4. Built-in executors: `HumanExecutor` and `ClaudeLightExecutor`

## Setup

In [ ]:
import tempfile
import numpy as np
import matplotlib.pyplot as plt

from folio.api import Folio
from folio.core.config import TargetConfig
from folio.core.schema import InputSpec, OutputSpec
from folio.core.observation import Observation
from folio.core.project import Project
from folio.executors import Executor, HumanExecutor, ClaudeLightExecutor

In [ ]:
# Create a temporary database
db_path = tempfile.mktemp(suffix=".db")
folio = Folio(db_path=db_path)
print(f"Database: {db_path}")

## What is an Executor?

An **Executor** is a class that takes suggested input values from Folio and
returns an `Observation` with the measured outputs. This abstraction allows:

- **Human-in-the-loop**: Prompt users to run experiments manually (`HumanExecutor`)
- **Autonomous operation**: Call APIs or robots automatically (`ClaudeLightExecutor`)
- **Simulation**: Test optimization strategies with synthetic functions

The Executor interface is simple:
```python
class Executor(ABC):
    def execute(self, suggestion: dict, project: Project) -> Observation:
        """Run experiment with given inputs, return observation."""
```

## Creating a Custom Executor

Let's create a **SimulatorExecutor** that runs a synthetic function.
This is useful for testing optimization strategies before deploying to real experiments.

In [ ]:
class SimulatorExecutor(Executor):
    """Executor that simulates experiments using a synthetic function.
    
    Parameters
    ----------
    func : callable
        Function that takes **inputs and returns dict of outputs.
    noise_std : float
        Standard deviation of Gaussian noise to add to outputs.
    """
    
    def __init__(self, func, noise_std: float = 0.0):
        self.func = func
        self.noise_std = noise_std
    
    def _run(self, suggestion: dict, project: Project) -> Observation:
        """Run the synthetic function with given inputs."""
        # Call the synthetic function
        outputs = self.func(**suggestion)
        
        # Add noise if specified
        if self.noise_std > 0:
            outputs = {
                k: v + np.random.normal(0, self.noise_std)
                for k, v in outputs.items()
            }
        
        return Observation(
            project_id=project.id,
            inputs=suggestion,
            inputs_suggested=suggestion,
            outputs=outputs,
        )

## Define a Synthetic Function

We'll optimize a 2D quadratic with optimum at (7, 3).

In [ ]:
def branin_like(x1: float, x2: float) -> dict:
    """A modified Branin-like function with known optimum.
    
    Optimum at (7, 3) with value 100.
    """
    value = 100 - (x1 - 7)**2 - (x2 - 3)**2
    return {"yield": value}

# True optimum for reference
TRUE_OPTIMUM = {"x1": 7.0, "x2": 3.0}
TRUE_YIELD = 100.0

# Test the function
print(f"At optimum: {branin_like(**TRUE_OPTIMUM)}")
print(f"At (0, 0): {branin_like(0, 0)}")

## Create Project and Executor

In [ ]:
# Create project
folio.create_project(
    name="executor_demo",
    inputs=[
        InputSpec("x1", "continuous", bounds=(0.0, 10.0)),
        InputSpec("x2", "continuous", bounds=(0.0, 10.0)),
    ],
    outputs=[OutputSpec("yield")],
    target_configs=[TargetConfig(objective="yield", objective_mode="maximize")],
)

# Create executor with slight noise
executor = SimulatorExecutor(branin_like, noise_std=0.5)
print("Project and executor created!")

## Using `folio.execute()` for Automated Loops

The `execute()` method automates the optimization loop:
1. Get suggestion via `suggest()`
2. Run experiment via `executor.execute()`
3. Record observation in database
4. Repeat for `n_iter` iterations

This replaces the manual loop we saw in earlier demos!

In [ ]:
# Run 15 iterations of automated optimization
observations = folio.execute(
    project_name="executor_demo",
    n_iter=15,
    executor=executor,
)

print(f"Completed {len(observations)} experiments!")

## Analyze Results

In [ ]:
# Get all observations
all_obs = folio.get_observations("executor_demo")
yields = [obs.outputs["yield"] for obs in all_obs]

# Find best
best_idx = np.argmax(yields)
best_obs = all_obs[best_idx]

print(f"Best result: yield={yields[best_idx]:.2f}")
print(f"  at x1={best_obs.inputs['x1']:.2f}, x2={best_obs.inputs['x2']:.2f}")
print(f"\nTrue optimum: yield={TRUE_YIELD} at x1={TRUE_OPTIMUM['x1']}, x2={TRUE_OPTIMUM['x2']}")

In [ ]:
# Plot optimization progress
running_best = np.maximum.accumulate(yields)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))

# Left: optimization progress
ax1.plot(range(1, len(yields) + 1), running_best, "b-o", label="Best found")
ax1.axhline(TRUE_YIELD, color="r", linestyle="--", label="True optimum")
ax1.set_xlabel("Iteration")
ax1.set_ylabel("Yield")
ax1.set_title("Optimization Progress")
ax1.legend()
ax1.grid(True, alpha=0.3)

# Right: sampled points
x1_vals = [obs.inputs["x1"] for obs in all_obs]
x2_vals = [obs.inputs["x2"] for obs in all_obs]
sc = ax2.scatter(x1_vals, x2_vals, c=range(len(all_obs)), cmap="viridis", s=60)
ax2.scatter([TRUE_OPTIMUM["x1"]], [TRUE_OPTIMUM["x2"]], c="red", marker="*", s=200, label="True optimum")
ax2.set_xlabel("x1")
ax2.set_ylabel("x2")
ax2.set_title("Sampled Points")
ax2.set_xlim(0, 10)
ax2.set_ylim(0, 10)
ax2.legend()
plt.colorbar(sc, ax=ax2, label="Iteration")

plt.tight_layout()
plt.show()

## Error Handling with `stop_on_error`

The `execute()` method supports graceful error handling. If an experiment fails,
you can choose to:
- `stop_on_error=True` (default): Stop immediately and raise the error
- `stop_on_error=False`: Log the error and continue with remaining iterations

In [ ]:
class FlakyExecutor(Executor):
    """An executor that occasionally fails (for demo purposes)."""
    
    def __init__(self, func, fail_prob: float = 0.3):
        self.func = func
        self.fail_prob = fail_prob
    
    def _run(self, suggestion: dict, project: Project) -> Observation:
        if np.random.random() < self.fail_prob:
            raise RuntimeError("Simulated equipment failure!")
        
        outputs = self.func(**suggestion)
        return Observation(
            project_id=project.id,
            inputs=suggestion,
            outputs=outputs,
        )

In [ ]:
# Create a new project for flaky executor demo
folio.create_project(
    name="flaky_demo",
    inputs=[
        InputSpec("x1", "continuous", bounds=(0.0, 10.0)),
        InputSpec("x2", "continuous", bounds=(0.0, 10.0)),
    ],
    outputs=[OutputSpec("yield")],
    target_configs=[TargetConfig(objective="yield", objective_mode="maximize")],
)

flaky_executor = FlakyExecutor(branin_like, fail_prob=0.3)

# Run with stop_on_error=False to continue despite failures
np.random.seed(42)  # For reproducibility
obs_list = folio.execute(
    project_name="flaky_demo",
    n_iter=10,
    stop_on_error=False,  # Continue despite errors
    executor=flaky_executor,
)

print(f"Attempted 10 experiments, {len(obs_list)} succeeded")

## Using `wait_between_runs`

For rate-limited APIs or experiments that need cooldown time, use `wait_between_runs`:

In [ ]:
import time

# Create another project
folio.create_project(
    name="wait_demo",
    inputs=[InputSpec("x", "continuous", bounds=(0.0, 10.0))],
    outputs=[OutputSpec("y")],
    target_configs=[TargetConfig(objective="y", objective_mode="maximize")],
)

def simple_func(x):
    return {"y": -(x - 5)**2 + 25}

simple_executor = SimulatorExecutor(simple_func)

start_time = time.time()
folio.execute(
    project_name="wait_demo",
    n_iter=3,
    wait_between_runs=0.5,  # Wait 0.5 seconds between runs
    executor=simple_executor,
)
elapsed = time.time() - start_time

print(f"3 iterations with 0.5s wait took {elapsed:.2f}s (expected ~1.5s of wait time)")

## Built-in Executors

Folio provides two built-in executors:

### HumanExecutor
Prompts a human to run experiments manually and enter results:
```python
from folio.executors import HumanExecutor

executor = HumanExecutor()
# When execute() is called, user is prompted:
#   "Suggested inputs: {'temperature': 80.0}"
#   "Enter actual temperature: "
#   "Enter yield: "
#   "Did experiment fail? [y/n]: "
```

### ClaudeLightExecutor
Calls the Claude-Light API for fully autonomous experiments:
```python
from folio.executors import ClaudeLightExecutor

executor = ClaudeLightExecutor(api_url="https://claude-light.cheme.cmu.edu/api")
# Sends RGB values to API, receives measured values back
```

## Using `build_executor()` for Quick Setup

For built-in executors, you can use `folio.build_executor()` to instantiate and cache them:

In [ ]:
# Build and cache a HumanExecutor
human_executor = folio.build_executor("human")
print(f"Built executor: {type(human_executor).__name__}")
print(f"Cached in folio.executor: {folio.executor is human_executor}")

# Switch to ClaudeLightExecutor
claude_executor = folio.build_executor("claude_light")
print(f"\nSwitched to: {type(claude_executor).__name__}")
print(f"Cached executor updated: {folio.executor is claude_executor}")

## HumanExecutor Demo

The `HumanExecutor` prompts users to enter actual input values and output measurements
via the command line. This is useful for manual experiments where you need to record
what was actually done (which may differ from the suggestion).

In [ ]:
# Create a project for human executor demo
folio.create_project(
    name="human_demo",
    inputs=[
        InputSpec("temperature", "continuous", bounds=(50.0, 150.0)),
        InputSpec("pressure", "continuous", bounds=(1.0, 10.0)),
    ],
    outputs=[OutputSpec("yield")],
    target_configs=[TargetConfig(objective="yield", objective_mode="maximize")],
)

# Run one iteration with HumanExecutor
# You will be prompted to enter:
#   1. Actual temperature (what you really set)
#   2. Actual pressure (what you really set)
#   3. The measured yield
#   4. Whether the experiment failed
#   5. Any notes
#   6. A tag (optional)
human_executor = HumanExecutor()
obs_list = folio.execute(
    project_name="human_demo",
    n_iter=1,
    executor=human_executor,
)

print(f"\nRecorded observation:")
print(f"  Inputs: {obs_list[0].inputs}")
print(f"  Outputs: {obs_list[0].outputs}")
print(f"  Failed: {obs_list[0].failed}")

## Summary

Executors enable **automated closed-loop optimization** in Folio:

1. **Create custom executors** by subclassing `Executor` and implementing `_run()`
2. **Use `folio.execute()`** to automate the suggest → execute → record loop
3. **Handle errors gracefully** with `stop_on_error=False`
4. **Control pacing** with `wait_between_runs`
5. **Use built-in executors** for human-in-the-loop or Claude-Light integration

This pattern makes it easy to:
- Test optimization strategies with simulators
- Deploy to real experiments with minimal code changes
- Run fully autonomous optimization campaigns

In [ ]:
# Cleanup
folio.delete_project("executor_demo")
folio.delete_project("flaky_demo")
folio.delete_project("wait_demo")
folio.delete_project("human_demo")
print("Demo complete!")